In [1]:
# Author: Xiaomeng Huang
# Date:   Aug. 2, 2018
#
# Solves the fifth test case in Cartesian coordinates on the sphere from: Williamson, et. al. "A Standard Test Set for Numerical Approximations to the Shallow Water Equations in Spherical Geometry",  J. Comput. Phys., 102 , 211-224, 1992.
#
# For details with regard to RBF-FD implemetation of the above test case, see Flyer et al., A guide to RBF-generated finite differences for nonlinear transport: ' Shallow water simulations on a sphere, J. Comput. Phys. 231 (2012) 4078?095

import tensorflow as tf
import numpy as np
import math as math
import matplotlib.pyplot as plt 
import matplotlib.tri as tri 
from scipy.spatial import cKDTree 
import time

In [2]:
#=============================Define Parameters==============================      
# size of RBF-FD stencil
fdsize= 0
nfile =np.loadtxt("md/md002.00009")
#nfile =np.loadtxt("md/md019.00400")
#file  =np.loadtxt("md/md059.03600")
N = nfile.shape[0]
# time step, needs to be in seconds
dt    = 0.0
#amount of hyperviscosity applied, multiplies Laplacian^order
gamma = -2.97e-16

learning_rate = 0.0 
beta1 = 0.0
weight = 0.0
if   N == 9:
    fdsize = 5 ; gamma = -2.97e-15 ; dt = 2000.0 ; learning_rate = 2e-6 ;  beta1 = 1e-1 ; weight1 = 2e6 ; weight2=1e-3
elif N == 400:
    fdsize = 31; gamma = -2.97e-15 ; dt = 2000.0 ; learning_rate = 1e-7 ;  beta1 = 1e-1 ; weight1 = 2e6 ; weight2=1e-3
elif N == 3600:
    fdsize = 31; gamma = -2.97e-15 ; dt = 2000.0 ; learning_rate = 4e-7 ;  beta1 = 1e-1 ; weight1 = 2e6 ; weight2=2e-3
elif N == 4900:
    fdsize = 31; gamma = -2.97e-15 ; dt = 1000.0 ; learning_rate = 1e-7 ;  beta1 = 1e-1 ; weight1 = 2e6 ; weight2=1e-3
elif N == 6400:
    fdsize = 31; gamma = -2.97e-16 ; dt = 1000.0 ; learning_rate = 1e-7 ;  beta1 = 1e-1 ; weight1 = 2e6 ; weight2=1e-3

# ending time, needs to be in days
tend  = 1
# power of Laplacian, L^order
order = 4
# dimension of stencil, on sphere dim=2
dim   = 2
# controls width of Gaussian RBF
ep    = 2.0

#gamma =0 
# Parameters for the mountain:
lam_c= -np.pi/2;
thm_c= np.pi/6;
mR   = np.pi/9;
hm0  = 2000.0;
# Angle of rotation measured from the equator.
alpha = 0.0
# Speed of rotation in meters/second
u0    = 20.0
# Mean radius of the earth (meters).
a     = 6.37122e6
# Rotation rate of the earth (1/seconds).
omega = 7.292e-5
g     = 9.80616
# Initial condition for the geopotential field (m^2/s^2).
gh0   = g*5960
# Set to nplt=1 if you want to plot results at different time-steps.
ndsply = 1
nplt   = 0

In [61]:
def setupT5(nfile):
    N = nfile.shape[0]      
    x = nfile[:,0].reshape((N,1))
    y = nfile[:,1].reshape((N,1))
    z = nfile[:,2].reshape((N,1))   
    X = nfile[:,0:3]
    la, th, r = cart2sph(x, y, z)
    
    p_u = np.hstack((1-x**2,   -x*y  ,  -x*z  ))
    p_v = np.hstack(( -x*y  , 1-y**2 ,  -y*z  ))
    p_w = np.hstack(( -x*z  ,  -y*z  , 1-z**2))    
    P = np.stack((p_u, p_v, p_w), 2)
    # Coriolis force.
    f = 2*omega*(-x*np.sin(alpha) + z*np.cos(alpha))
    # Computer the profile of the mountain (multiplied by gravity)
    r2 = (la-lam_c)**2+(th-thm_c)**2
    id = r2 < mR**2
    ghm = g * hm0 * (1 - np.sqrt(r2*id)/mR) * id

    return X, la, th ,r, P, f, ghm

def cart2sph(a,b,c):
    ab = a**2 + b**2
    r = np.sqrt(ab + c**2)            # radius
    elev = np.arctan2(c, np.sqrt(ab)) # elevation
    az = np.arctan2(b, a)             # azimuth
    return az, elev, r

# Setup tc5 case
print(">>Setup testcase ...")
X, la, th, r, P, f, ghm = setupT5(nfile)

>>Setup testcase ...


In [65]:
# Calculate the RBF-FD differentiation matrices. Requires kd-tree code.
def rbfmatrix_fd_hyper(X, ep, fdsize, order, dim):
    # get the number of points  
    N = X.shape[0]
    #Query the kd-tree for nearest neighbors
    tree = cKDTree(X)
    dist, index = tree.query(X, k=fdsize)
    #print('dist=\n', dist,'\n------index=\n',index)

    weightsDx =  np.zeros(N * fdsize, dtype=np.float64)
    weightsDy =  np.zeros(N * fdsize, dtype=np.float64)
    weightsDz =  np.zeros(N * fdsize, dtype=np.float64)
    weightsL  =  np.zeros(N * fdsize, dtype=np.float64)

    imat   =  np.zeros([1,3], dtype=np.float64)
    ind_i  =  np.zeros(N * fdsize, dtype=np.int64)
    ind_j  =  np.zeros(N * fdsize, dtype=np.int64)

    A      =  np.ones((fdsize+1, fdsize+1), dtype=np.float64)
    A[fdsize,fdsize] = 0
    B      =  np.zeros(fdsize+1, dtype=np.float64)
    for j in range(N):
        imat = index[j,:]
        ind_i[j*fdsize : (j+1)*fdsize] = j
        ind_j[j*fdsize : (j+1)*fdsize] = imat

        dp = np.dot(nfile[j,0], nfile[imat,0]) +  \
             np.dot(nfile[j,1], nfile[imat,1]) +  \
             np.dot(nfile[j,2], nfile[imat,2])
        
        x1 =  nfile[imat,0].reshape((fdsize,1))
        x2 =  nfile[imat,1].reshape((fdsize,1))
        x3 =  nfile[imat,2].reshape((fdsize,1))
        rd2   = np.maximum(0, 2*(1-np.dot(x1, x1.T) - np.dot(x2, x2.T) - np.dot(x3, x3.T)))
        rd2v  = rd2[:,0]
        
        A[0:fdsize, 0:fdsize] = rbf(ep, rd2)
        B[0:fdsize] = (np.dot(nfile[j,0], dp) - nfile[imat,0]) * drbf(ep, rd2v)
        weights = np.dot(np.linalg.inv(A), B)
        weightsDx[j*fdsize : (j+1)*fdsize] =  weights[0 : fdsize]

        B[0:fdsize] = (np.dot(nfile[j,1], dp) - nfile[imat,1]) * drbf(ep, rd2v)
        weights = np.dot(np.linalg.inv(A), B)
        weightsDy[j*fdsize : (j+1)*fdsize] =  weights[0 : fdsize]

        B[0:fdsize] = (np.dot(nfile[j,2], dp) - nfile[imat,2]) * drbf(ep, rd2v)
        weights = np.dot(np.linalg.inv(A), B)
        weightsDz[j*fdsize : (j+1)*fdsize] =  weights[0 : fdsize]

        B[0:fdsize] = ep**(2*order) * hyper(ep**2 * rd2v, dim, order) * np.exp(-ep**2 * rd2v)

        weights = np.dot(np.linalg.inv(A), B)
        weightsL[j*fdsize : (j+1)*fdsize] =  weights[0 : fdsize]

    indices=np.concatenate((np.expand_dims(ind_i, 1), np.expand_dims(ind_j, 1)), 1)
    #print('indices=\n', indices)

    return indices, weightsDx, weightsDy, weightsDz, weightsL

def rbf(ep, rd2):
    return np.exp(-ep**2 * rd2)

def drbf(ep, rd2):
    return -2 * ep**2 * rbf(ep, rd2)

def hyper(ep2r2, d, k):
    n = len(ep2r2)
    P = np.zeros((n,k+1), dtype=np.float64)
    P[:,0] = 1
    P[:,1] = 4*ep2r2 - 2 * d
    for j in range(2,k+1):
        P[:,j] = 4*(ep2r2-2*(j+1)-d/2+4)*P[:,j-1] - 8*(j-1)*(2*(j+1)+d-6)*P[:,j-2]
    return P[:,k]

print(">>Compute RBF matrix ...")
indices, weightsDx, weightsDy, weightsDz, weightsL =\
    rbfmatrix_fd_hyper(X, ep, fdsize, order, dim)

>>Compute RBF matrix ...
9
